In [1]:
%load_ext rustdef

load rustdef


In [2]:
%rustdef depends ndarray

In [3]:
%%rustdef
use rustdef::Numpy;
use ndarray::{ArrayD, ArrayViewMutD};
use pyo3;
use pyo3::prelude::{pymodule, PyModule, PyResult, Python};

// immutable example
#[pyfunction]
fn axpy(a: f64, x: Numpy<f64>, y: Numpy<f64>) -> PyResult<Numpy<f64>> {
    let x: ArrayD<f64> = x.into_array();
    let y: ArrayD<f64> = y.into_array();
    let z = a * x + y;
    Ok(Numpy(z))
}

In [4]:
import numpy as np

In [5]:
axpy(2.4, np.array([1.0, 3.2, 2.9]), np.array([2.3, 10.9, -2]))

array([ 4.7 , 18.58,  4.96])